## Challenge: Exercise 4

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### To match searches with bookings

In [2]:
import bz2
files_path = "/home/dsc/Repositories/Master-in-Data-Science/Data/"

##### Loading and formatting searches

In [119]:
bookings_file = bz2.BZ2File(files_path+"bookings.csv.bz2")
bookings_chk= pd.read_csv(bookings_file, sep='^',chunksize=5e5,usecols=['brd_time           '])

bookings_df= DataFrame([])
n=0
for chunk in bookings_chk:
    if n>-1:
        bookings_df = pd.concat([bookings_df,chunk],ignore_index=True)
        bookings_df = bookings_df.drop_duplicates()
    n += 1
    print n, len(bookings_df)
    if n > 200:
        break


1 105409
2 167848
3 167848
4 167848
5 167848
6 167848
7 167848
8 167848
9 167848
10 167848
11 167849
12 167849
13 167849
14 167849
15 167849
16 167849
17 167849
18 167849
19 167849
20 167849
21 167849


In [120]:
sorted = bookings_df['brd_time           '].copy()

In [121]:
sorted.sort()
sorted

43417     2012-03-02 12:15:00
166504    2012-07-28 17:10:00
133394    2012-09-01 12:15:00
101661    2012-09-04 01:00:00
73628     2012-10-01 19:20:00
117185    2012-10-03 12:45:00
126384    2012-10-04 02:10:00
58212     2012-10-09 19:30:00
79517     2012-10-21 12:00:00
93733     2012-11-19 19:40:00
74235     2012-11-27 12:45:00
10036     2012-12-04 02:35:00
150967    2012-12-04 18:35:00
129341    2012-12-06 13:10:00
129342    2012-12-07 15:05:00
76395     2012-12-09 01:50:00
17424     2012-12-10 19:05:00
33922     2012-12-11 21:50:00
26514     2012-12-12 16:20:00
26515     2012-12-13 14:35:00
33232     2012-12-16 05:50:00
77305     2012-12-18 07:20:00
67280     2012-12-18 17:45:00
167596    2012-12-19 01:50:00
42992     2012-12-21 19:55:00
73879     2012-12-22 06:42:00
26362     2012-12-22 12:30:00
64135     2012-12-23 15:00:00
17723     2012-12-27 15:10:00
69747     2012-12-27 18:05:00
                 ...         
140483    2014-11-18 15:30:00
138394    2014-11-20 05:15:00
103465    

In [51]:
searches_file = bz2.BZ2File(files_path+"searches.csv.bz2")
searches_df = pd.read_csv(searches_file, 
                          sep='^',
                          nrows=1e5,
                          usecols=['Origin','Destination','Seg1Date'],
                          parse_dates=['Seg1Date'])
searches_df.reset_index(inplace=True)

##### Loading and formatting bookings

In [62]:
bookings_file = bz2.BZ2File(files_path+"bookings.csv.bz2")
bookings_df = pd.read_csv(bookings_file, sep='^',nrows=11e5,usecols=['dep_port','arr_port','brd_time           ','pax'],parse_dates=['brd_time           '])
#bookings_df = bookings_df[100000:]
print len(bookings_df)

# Discarding blanks
bookings_df.dropna(inplace=True)

# Discarding "negative" bookings
bookings_df = bookings_df[bookings_df['pax'] >=1]
bookings_df.drop('pax', axis=1, inplace=True)

# Formatting bookings dataframe to match searches dataframe
bookings_df['Origin'] = bookings_df['dep_port'].apply(lambda s: s.strip())
bookings_df.drop('dep_port', axis=1, inplace=True)

bookings_df['Destination'] = bookings_df['arr_port'].apply(lambda s: s.strip())
bookings_df.drop('arr_port', axis=1, inplace=True)

bookings_df['Seg1Date'] = bookings_df['brd_time           '].apply(lambda d: pd.to_datetime(d.date()))
bookings_df.drop('brd_time           ', axis=1, inplace=True)

# Adding requested column "if search = booking"
bookings_df['Is_Booked'] = 1

1100000


##### Merging both dataframes

In [63]:
# Obtaining searches with bookings
merged_df = pd.merge(searches_df,bookings_df,how="left",on=['Origin','Destination','Seg1Date']).drop_duplicates()
merged_df['Is_Booked'].fillna(0,inplace=True)
merged_df.groupby('Is_Booked').count()
# merged_df.head()

,index,Origin,Destination,Seg1Date
Is_Booked,,,,
0,92400,92400,92400,92005
1,7600,7600,7600,7600
